In [1]:
from huggingface_hub import login
import os 
from dotenv import load_dotenv #importing env file


In [2]:
load_dotenv()
hf_token =os.environ["HF_TOKEN"]
login(hf_token, add_to_git_credential=True)



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
#importing src in order to import items 
import sys
import os

# Adding src to Python path
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from items import Item

from parallel_loader import ItemLoader

In [4]:
items = ItemLoader("Appliances").load_and_process_data()

Loading dataset Appliances


100%|██████████| 95/95 [00:17<00:00,  5.42it/s]


Completed Appliances with 27,047 datapoints in 0.4 mins


In [5]:
dataset_names=[
    "Automotive",
    "Electronics",
    "Office_Products",
    "Tools_and_Home_Improvement",
    "Cell_Phones_and_Accessories",
    "Toys_and_Games",
    "Appliances",
    "Musical_Instruments",
    "Health_and_Personal_Care.jsonl",
    "Video_Games" ,
    "Grocery_and_Gourmet_Food",
    "All_Beauty",
]


In [ ]:
items=[]
for dataset_name in dataset_names:
    loader = ItemLoader(dataset_name)
    items.extend(loader.load_and_process_data())

Loading dataset Automotive


meta_Automotive.jsonl:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

In [ ]:
print(f"Total No of items is :{len(items)}")